#Anwendung von Deep Learning-Verfahren in der Medizinischen Bildanalyse 
## Automatische Segmentierung des Rechten Ventrikels auf Kardio-MRT-Daten (MICCAI Challenge 2017)

# Imports 
Sry for the pain in the eyes


In [ ]:
# 1.   Install requiered modules
!pip install -U plotly
!pip install SimpleITK --quiet
!pip install --upgrade batchgenerators --quiet
!pip install opencv-python
!pip3 install medpy

In [ ]:
from tensorflow.keras.optimizers import Adam

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import natsort
import os
from batchgenerators.utilities.file_and_folder_operations import *
import SimpleITK as sitk


from multiprocessing import Pool


class CreateNumpyArr():
    def __init__(self,srcDir,destinationFolder,subfolders,norm_method=None):
        self.srcDir = srcDir
        self.destinationFolder = destinationFolder
        self.subfolders = subfolders
        self.norm_method = norm_method
        self.list_of_files = self.get_list_of_files()
        self.createNpArr()
        
    def createNpArr(self,num_threads =1):
        list_of_lists = self.list_of_files
        folderDst =self.destinationFolder
        maybe_mkdir_p(folderDst)
        patient_names = [i[0][0].split("/")[-1][0:10] for i in list_of_lists]
        p = Pool(processes=1)
        p.starmap(self.load_and_preprocess, zip(list_of_lists, patient_names, [folderDst] * len(list_of_lists)))
        p.close()
        p.join()
    def get_list_of_files(self):
        """
        returns a list of lists containing the filenames. The outer list contains all training examples. Each entry in the
        outer list is again a list pointing to the files of that training example in the following order:
        MRT, Segmentation
        :param srcDir: 
        :type srcDir: str 
        :return: lists of files
        """
        list_of_lists = []

        for type in self.subfolders:
            current_directory = os.path.join(self.srcDir, type)
            patients = subfolders(current_directory, join=False)
            for p in patients:
                image = []
                segmentation = []
                for file in os.listdir(os.path.join(current_directory,p,"image")):
                    image.append(os.path.join(current_directory,p,"image",file))
                for file in os.listdir(os.path.join(current_directory,p,"segmentation")):
                    segmentation.append(os.path.join(current_directory,p,"segmentation",file))
                
                image=natsort.natsorted(image)
                segmentation = natsort.natsorted(segmentation)
                list_of_lists.append([image,segmentation])


                # assert all((isfile(i) for i in this_case)), "some file is missing for patient %s; make sure the following " \
                #                                             "files are there: %s" % (p, str(this_case))
        print("Found %d patients" % len(list_of_lists))
        return list_of_lists



    def load_and_preprocess(self,case, patient_name, output_folder):
        """
        loads, preprocesses and saves a case
        This is what happens here:
        1) load all images and stack them to a 4d array
        2) crop to nonzero region, this removes unnecessary zero-valued regions and reduces computation time
        3) normalize the nonzero region with its mean and standard deviation
        4) save 4d tensor as numpy array. Also save metadata required to create niftis again (required for export
        of predictions)
        :param case:
        :param patient_name:
        :return:
        """
        
        # load SimpleITK Images
        imgs_sitk = [sitk.ReadImage(i) for i in case]
           # get some metadata
        spacing = imgs_sitk[0].GetSpacing()
        
        direction = imgs_sitk[0].GetDirection()
        origin = imgs_sitk[0].GetOrigin()

        # get pixel arrays from SimpleITK images
        imgs_npy = [sitk.GetArrayFromImage(i) for i in imgs_sitk]
        
        
        # I removed the unlabeled mask because i am pretty sure that those are showing the hear
        # When i am wrong, its not a big deal for RV Segmentation, because there remain enough
        # unlabled masks after setting Myo an lv to zero
        # idx_to_delete = []
        
       
        # for index,arr in enumerate(imgs_npy[1]):
        #         if not arr.any():
        #             idx_to_delete.append(index)   
        

        # # Removing images without labled mask
        # imgs_npy[0]= np.delete(imgs_npy[0],idx_to_delete,0)
        # # Removing unlabeled masks
        # imgs_npy[1] =np.delete(imgs_npy[1],idx_to_delete,0)
        # # get some metadata
        
        original_shape = imgs_npy[0].shape

        # now stack the images into one 4d array, cast to float because we will get rounding problems if we don't
        imgs_npy = np.concatenate([i[None] for i in imgs_npy]).astype(np.float32)

        # now find the nonzero region and crop to that
        nonzero = [np.array(np.where(i != 0)) for i in imgs_npy]
        nonzero = [[np.min(i, 1), np.max(i, 1)] for i in nonzero]
        nonzero = np.array([np.min([i[0] for i in nonzero], 0), np.max([i[1] for i in nonzero], 0)]).T
        # nonzero now has shape 3, 2. It contains the (min, max) coordinate of nonzero voxels for each axis

        # now crop to nonzero
        imgs_npy = imgs_npy[:,
                nonzero[0, 0] : nonzero[0, 1] + 1,
                nonzero[1, 0]: nonzero[1, 1] + 1,
                nonzero[2, 0]: nonzero[2, 1] + 1,
                ]

       
        # create a thorax mask that we use for normalization
        nonzero_masks = [i != 0 for i in imgs_npy[:-1]]
        thorax_mask = np.zeros(imgs_npy.shape[1:], dtype=bool)
        for i in range(len(nonzero_masks)):
            thorax_mask = thorax_mask | nonzero_masks[i]

        # minmax normalization using percentiles to be robust against outliers
        if self.norm_method == "min_max":
          for i in range(len(imgs_npy) - 1):
              percentile =np.percentile(imgs_npy.ravel(),99)
              imgs_npy[i][imgs_npy[i]>percentile] = percentile
              max = imgs_npy[i][thorax_mask].max()
              min = imgs_npy[i][thorax_mask].min()
              imgs_npy[i] = 0 if (max-min) ==0 else (imgs_npy[i] - min) / (max - min)
              imgs_npy[i][thorax_mask == 0] = 0
        if self.norm_method == "z_score":
          for i in range(len(imgs_npy) - 1):
                mean = imgs_npy[i][thorax_mask].mean()
                std = imgs_npy[i][thorax_mask].std()
                imgs_npy[i] = (imgs_npy[i] - mean) / (std + 1e-8)
                imgs_npy[i][thorax_mask == 0] = 0
        # now save as npz
        np.save(os.path.join(output_folder, patient_name + ".npy"), imgs_npy)

        metadata = {
        'spacing': spacing,
        'direction': direction,
        'origin': origin,
        'original_shape': original_shape,
        'nonzero_region': nonzero
        }

        save_pickle(metadata, join(output_folder, patient_name + ".pkl"))



In [ ]:
# 2.   Import needed modules
%matplotlib inline
import matplotlib.pyplot as plt
import numpy  as np
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
import matplotlib.pyplot as plt
import tensorflow as tf
import SimpleITK as sitk
import sys
import os
import pandas as pd
import plotly.io as pio
pd.options.plotting.backend = "plotly"
pio.renderers.default = "colab"
from batchgenerators.augmentations.crop_and_pad_augmentations import crop
from batchgenerators.augmentations.utils import pad_nd_image
import matplotlib.pyplot as plt
import cv2
import math 
import plotly.express as px
import plotly.graph_objects as go
from tensorflow import keras

In [ ]:
from time import time
from batchgenerators.augmentations.crop_and_pad_augmentations import crop
from batchgenerators.dataloading.multi_threaded_augmenter import MultiThreadedAugmenter
from batchgenerators.examples.brats2017.config import brats_preprocessed_folder, num_threads_for_brats_example
from batchgenerators.transforms.abstract_transforms import Compose
from batchgenerators.utilities.data_splitting import get_split_deterministic
from batchgenerators.utilities.file_and_folder_operations import *
import numpy as np
from batchgenerators.dataloading.data_loader import DataLoader
from batchgenerators.augmentations.utils import pad_nd_image
from batchgenerators.transforms.spatial_transforms import SpatialTransform_2, MirrorTransform
from batchgenerators.transforms.color_transforms import BrightnessMultiplicativeTransform, GammaTransform
from batchgenerators.transforms.noise_transforms import GaussianNoiseTransform, GaussianBlurTransform


class ImageLoader2D(DataLoader):
    def __init__(self, data, batch_size, patch_size,segmentation_value, num_threads_in_multithreaded=1,seed_for_shuffle=1234, return_incomplete=False,
                 shuffle=True,infinite = True):
        """
        data must be a list of patients as returned by get_list_of_patients (and split by get_split_deterministic)
        patch_size is the spatial size the retured batch will have
        """
       
        super().__init__(data, batch_size, num_threads_in_multithreaded, seed_for_shuffle, return_incomplete, shuffle,
                         True)
        self.segmentation_value =segmentation_value
        self.patch_size = patch_size
        self.indices = list(range(len(data)))
        self.list_of_used_indices ={}
        
    @staticmethod
    def load_patient(patient):
        data = np.load(patient + ".npy", mmap_mode="r")
        # metadata = load_pickle(patient + ".pkl")
        return data#, metadata

    def generate_train_batch(self):
        # DataLoader has its own methods for selecting what patients to use next, see its Documentation
        idx = self.get_indices()
        patients_for_batch = [self._data[i] for i in idx]
       
        # initialize empty array for data and seg
        data = np.zeros((self.batch_size,4,*self.patch_size), dtype=np.float32)
        seg = np.zeros((self.batch_size,4,*self.patch_size), dtype=np.float32)

        metadata = []
        patient_names = []

        # iterate over patients_for_batch and include them in the batch
        for i, j in enumerate(patients_for_batch):
            patient_data = self.load_patient(j)

            
            slice_idx = np.random.choice(patient_data.shape[1])
            patient_data = patient_data[:, slice_idx]
            
            # this will only pad patient_data if its shape is smaller than self.patch_size
            patient_data = pad_nd_image(patient_data, self.patch_size)

            # now random crop to self.patch_size
            # crop expects the data to be (b, c, x, y, z) but patient_data is (c, x, y, z) so we need to add one
            # dummy dimension in order for it to work (@Todo, could be improved)
            # if len(patient_data.shape)< 5:
              #Todo resize instead of cropping
            patient_data, patient_seg = crop(patient_data[:-1][None], patient_data[-1:][None], (4,)+self.patch_size, crop_type="random")
            

            # patient_seg = patient_data[-1:][None]
            # patient_data =patient_data[:-1][None]
            data[i] = patient_data[0]
            seg[i] = patient_seg[0]
            seg[i][seg[i]!= self.segmentation_value] =0
            
        return {'data': data, 'seg':seg}



In [ ]:
import numpy as np
import cv2
import os
from batchgenerators.utilities.file_and_folder_operations import *
class Preprocess():
  def __init__(self,list_of_patients,patch_size,threshold,destination_folder):
    self.resample(list_of_patients,patch_size,threshold,destination_folder)
  @staticmethod
  def load_patient(patient):
      data = np.load(patient + ".npy", mmap_mode="r")
      metadata = load_pickle(patient + ".pkl")
      return data, metadata
  
  def resample(self,list_of_patients,new_shape,threshold,destination_folder):
   
    for i,patient in enumerate(list_of_patients):
      data, metadata = self.load_patient(patient)
      patient_name = list_of_patients[i].split("/")[-1][0:10]
      image = data[0]
      segmentation = data[1]
      seg = np.zeros((segmentation.shape[0],4,new_shape[1],new_shape[0]),dtype=np.float32)
      img = np.zeros((image.shape[0],4,new_shape[1],new_shape[0]),dtype=np.float32)
      # Resizing of the image creation of binary segmentation channels
      for j,mrt_slice in enumerate(segmentation):
        for index,segmentation_value in enumerate([0,1,2,3]):
          seg_slice = mrt_slice == segmentation_value
          seg[j][index] = cv2.resize(seg_slice.astype(np.float32),new_shape, interpolation = cv2.INTER_AREA)
          seg[j][index][ seg[j][index]<threshold] =0
          seg[j][index][ seg[j][index]>=threshold] =1
        img[j] = cv2.resize(image[j], new_shape,interpolation = cv2.INTER_AREA)
        
      np_array = np.concatenate([img[None],seg[None]],axis =0).astype(np.float32)
      
      np.save(os.path.join(destination_folder, patient_name + ".npy"), np_array)
  

In [ ]:
import os
import numpy as np
import shutil
import natsort
import os
from batchgenerators.utilities.file_and_folder_operations import *
import numpy as np

try:
    import SimpleITK as sitk
except ImportError:
    print("You need to have SimpleITK installed to run this example!")
    raise ImportError("SimpleITK not found")
from multiprocessing import Pool



class DivideData():
    
    
    def __init__(self,source_directory,destination_directory,subdirectories):
        """Initializes a divideData object that reads data from a the srcDir and saves it in provided folder structure
        
        :param source_directory: path to source folder where the raw data is placed in one folder
        :type source_dircetory: str
        
        :param destination_directory: path to root folder of the subfolders
        :type destination_directory: str
        
        :param subfolders: list of folder names in wich to subdivide the data
        :type subfolders: list 
        """
        self.load_files(source_directory)
        self.divide_data(source_directory,destination_directory,subdirectories)


    def load_files(self,source_directory):
        """Loads files from subfolder to self.files"""
        self.files = os.listdir(source_directory)
        self.files.sort()

    def divide_data(self,source_directory,destination_directory,subdirectories):
        '''Divides data from self.files into provided folderstructure (self.subfolders)'''
        files = natsort.natsorted(self.files)

        for i,pathologie in enumerate(subdirectories):
            pathologie_folder = os.path.join(destination_directory,pathologie)
            for index,patient in enumerate(files):
                    if int(patient[7:10]) > (i+1)*20:
                        files = files[index:]
                        break
                    if patient[-8:-5] == "msk":
                        subfolder ="segmentation"
                    else:
                        subfolder ="image"
                    patient_folder=os.path.join(pathologie_folder,patient[0:10])
                    patient_subfolder=os.path.join(patient_folder,subfolder)
                    if not os.path.exists(patient_subfolder): 
                            os.makedirs(patient_subfolder)
                    shutil.copy(src=os.path.join(source_directory,patient),dst=patient_subfolder)

In [ ]:
from keras.models import Input, Model
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
"""
Adapted implementation from: https://github.com/pietz/unet-keras/blob/master/unet.py
"""
'''
U-Net: Convolutional Networks for Biomedical Image Segmentation
(https://arxiv.org/abs/1505.04597)
---
img_shape: (height, width, channels)
out_ch: number of output channels
start_ch: number of channels of the first conv
depth: zero indexed depth of the U-structure
increasing_rate: rate at which the conv channels will increase
activation: activation function after convolutions
dropout: amount of dropout in the contracting part
batchnorm: adds Batch Normalization if true
maxpool: use strided conv instead of maxpooling if false
upconv: use transposed conv instead of upsamping + conv if false
residual: add residual connections around each conv block if true
'''

def convolution_block(inputs, dimension, activation, batchnorm, resudial, dropout=0):
	n = Conv2D(dimension, 3, activation=activation, padding='same')(inputs)
	n = BatchNormalization()(n) if batchnorm else n
	n = Dropout(dropout)(n) if dropout else n
	n = Conv2D(dimension, 3, activation=activation, padding='same')(n)
	n = BatchNormalization()(n) if batchnorm else n
	return Concatenate()([inputs, n]) if resudial else n

def level_block(inputs, dimension, depth, increasing_rate, activation, dropout, batchnorm, maxpooling, up, resudial):
	if depth > 0:
		n = convolution_block(inputs, dimension, activation, batchnorm, resudial)
		m = MaxPooling2D()(n) if maxpooling else Conv2D(dimension, 3, strides=2, padding='same')(n)
		m = level_block(m, int(increasing_rate*dimension), depth-1, increasing_rate, activation, dropout, 
    batchnorm, maxpooling, up, resudial)
		if up:
			m = UpSampling2D()(m)
			m = Conv2D(dimension, 2, activation=activation, padding='same')(m)
		else:
			m = Conv2DTranspose(dimension, 3, strides=2, activation=activation, padding='same')(m)
		n = Concatenate()([n, m])
		m = convolution_block(n, dimension, activation, batchnorm, resudial)
	else:
		m = convolution_block(inputs, dimension, activation, batchnorm, resudial, dropout)
	return m

def UNet(img_shape, out_ch=1, start_ch=64, depth=4, increasing_rate=2., activation='relu', 
		 dropout=0.5, batchnorm=True, maxpool=True, upconv=True, residual=False,act_fun_final_conv='sigmoid'):
	i = Input(shape=img_shape)
	o = level_block(i, start_ch, depth, increasing_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
	o = Conv2D(out_ch, 1, activation=act_fun_final_conv)(o)
	return Model(inputs=i, outputs=o)

In [ ]:



from time import time
from batchgenerators.augmentations.crop_and_pad_augmentations import crop
from batchgenerators.dataloading.multi_threaded_augmenter import MultiThreadedAugmenter
from batchgenerators.examples.brats2017.config import brats_preprocessed_folder, num_threads_for_brats_example
from batchgenerators.transforms.abstract_transforms import Compose
from batchgenerators.utilities.data_splitting import get_split_deterministic
from batchgenerators.utilities.file_and_folder_operations import *
import numpy as np
import natsort
from batchgenerators.utilities.file_and_folder_operations import *
import numpy as np
from sklearn.model_selection import KFold
from batchgenerators.transforms.spatial_transforms import SpatialTransform_2, MirrorTransform
from batchgenerators.transforms.sample_normalization_transforms import RangeTransform, CutOffOutliersTransform
from batchgenerators.transforms.color_transforms import BrightnessMultiplicativeTransform, GammaTransform
from batchgenerators.transforms.noise_transforms import GaussianNoiseTransform, GaussianBlurTransform
def get_list_of_patients(preprocessed_data_folder):
        npy_files = subfiles(preprocessed_data_folder, suffix=".npy", join=True)
        # remove npy file extension
        patients = [i[:-4] for i in npy_files]
        return natsort.natsorted(patients)


def get_split_deterministic(all_keys, fold=0, num_splits=5, random_state=12345):
    """
    Splits a list of patient identifiers (or numbers) into num_splits folds and returns the split for fold fold.
    :param all_keys:
    :param fold:
    :param num_splits:
    :param random_state:
    :return:
    """
    all_keys_sorted = np.sort(list(all_keys))
    splits = KFold(n_splits=num_splits, shuffle=True, random_state=random_state)
    for i, (train_idx, test_idx) in enumerate(splits.split(all_keys_sorted)):
        if i == fold:
            train_keys = np.array(all_keys_sorted)[train_idx]
            test_keys = np.array(all_keys_sorted)[test_idx]
            break
    return train_keys, test_keys
def get_train_transform(patch_size):
    # we now create a list of transforms. These are not necessarily the best transforms to use for BraTS, this is just
    # to showcase some things
    tr_transforms = []

    # Elastic deform, rotation sacling, and randomcropping with a probabilty of 0.2 each 
    tr_transforms.append(
          SpatialTransform_2(
            patch_size, [i // 2 for i in patch_size],
            do_elastic_deform=True, deformation_scale=(0, 0.25),
            do_rotation=True,
            angle_x=(- 15 / 360. * 2 * np.pi, 15 / 360. * 2 * np.pi),
            angle_y=(- 15 / 360. * 2 * np.pi, 15 / 360. * 2 * np.pi),
            angle_z=(- 15 / 360. * 2 * np.pi, 15 / 360. * 2 * np.pi),
            do_scale=True, scale=(0.75, 1.25),
            border_mode_data='constant', border_cval_data=0,
            border_mode_seg='constant', border_cval_seg=0,
            order_seg=0, order_data=3,
            random_crop=True,
            p_el_per_sample=0.2, p_rot_per_sample=0.2, p_scale_per_sample=0.2
        )
    )
    tr_transforms.append(CutOffOutliersTransform())
    
    tr_transforms.append(RangeTransform((0, 1), True, data_key="data", label_key="seg"))


    tr_transforms = Compose(tr_transforms)
    return tr_transforms
def get_val_transform(patch_size):
  tr_transforms = []
  tr_transforms.append(CutOffOutliersTransform())
    
  tr_transforms.append(RangeTransform((0, 1), True, data_key="data", label_key="seg"))


  tr_transforms = Compose(tr_transforms)
  return tr_transforms


#Colab specific settings

> 1. Checking if GPU is enabled
> 2. Mounting Google Drive, to get the created python modules



In [ ]:
# # Check if GPU is enabled
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#Setting up python environment


1.   Install requiered modules
2.   Import needed modules
3.   Add bin to system path
4.   Import local modules from bin




In [ ]:
# 3.   Add bin to system path
root_dir=""
# append root_dir to systempath to make local imports possible
module_dir = os.path.join(root_dir,"bin")
sys.path.append(module_dir)

#Get the Data
1. Download the data
2. Unzip the data

In [ ]:
# 1. Download and unzip  the data

data_dir =  os.path.join(root_dir,"data")
if not os.path.isdir(data_dir):
  os.makedirs(data_dir)
  !wget -O data.html    https://heibox.uni-heidelberg.de/f/eef4cf141a4746d2bf45/?dl=1
  !unzip   -q 'data.html' -d "/data"

# Stratification of the data
1. Define directory paths
2. Divide data into the provided subfolders

In [ ]:
# 1. Define directory paths
# To have a leightweight implementation we only want to have the current batch in RAM. Therefore
# everything else is stored on the harddrive
raw_data_dir = os.path.join(root_dir,"/data/all")
destination_dir = os.path.join(root_dir,"/data/dividedData")
np_data_dir = os.path.join(root_dir,"/data/numpy")
np_preprocessed_data_dir = os.path.join(root_dir,"/data/np_preprocessed")
subfolder = ["DCM","HCM","MINF","NOR","RV"]

In [ ]:
# 2. Divide data into the provided dirStructure
# For every category ("DCM","HCM","MINF","NOR","RV") one folder is created where the corresponding nrrd files are stored
if not os.path.isdir(destination_dir):
  dataDivider = DivideData(raw_data_dir,destination_dir,subfolder)
else: 
  print("The directory: {} already exists, therefore the data Division is not\
  done.\n If you want to load the data again delete\
  the directory: {}".format(destination_dir,destination_dir))

#Preprocessing
1. Load data from nrrd into numpy Array
2.Get an overview of the data
*The implementation of the data preprocessing and augmentation is heavely based on the BraTS2017/2018 example provided by MIC-DKFZ: [Example Batchgenerators](https://github.com/MIC-DKFZ/batchgenerators/tree/master/batchgenerators/examples/brats2017)*

In [ ]:
# 1. Load data from nrrd into numpy Array
# With attached google drive this step sometimes takes ages
# We stack the slices for each patien into a 4 D image First dimension is image/segmnetation
# second dimension is the channel, 3rd and 4th the image/seg dimensions
# Also a min_max normalization is done for performance reasons (after data augmentation another one is done)
if not os.path.isdir(np_data_dir):
  numpyGenerator = CreateNumpyArr(destination_dir,np_data_dir,subfolder,"min_max")
else: 
  print("The directory: {} already exists, therefore the numpy array are not\
  created.\n If you want to load the data again delete\
  the directory: {}".format(np_data_dir,np_data_dir))

In [ ]:
# Patient 6 [0,11]
# Patient 1 [0,10]
# Patient 12 [20]
# Patient 21 [10]
# Patient 23 [8,17]
# Patient 26 [10]
# Patient 27 [10]
# Patint 28 [9, 19]
# Patient 29[11, 12, 21]
# Patiennt 30 [0, 10, 19]
# Patient 31 [10, 19]
# Patient 32[10]
# Pateint 33 [0, 10, 11, 19]
# plt.rcParams['figure.figsize'] = [40, 40]
# plt.subplot(2,2,1)
# plt.imshow(np.load("/content/drive/MyDrive/ADVMB/data/numpy/patient006.npy")[0][0])
# plt.subplot(2,2,2)
# plt.imshow(np.load("/content/drive/MyDrive/ADVMB/data/numpy/patient006.npy")[0][1])
# plt.subplot(2,2,3)
# plt.imshow(np.load("/content/drive/MyDrive/ADVMB/data/numpy/patient006.npy")[1][0])
# plt.subplot(2,2,4)
# plt.imshow(np.load("/content/drive/MyDrive/ADVMB/data/numpy/patient006.npy")[1][1])

## 2. Get an overview of the data




1.   Count total amount of slices (2D-images)
2.   Plot histogram on a subset of slices
3. Get maximum space
3. Resize images
3.   Get the image dimensions
4.   Plot histogram of image dimensions
5.   Choose the patch_size



In [ ]:
# 1. Count total amount of slices (2D-images)
number_of_total_slices = 0 
hist_list = []
slice_nr = 5
for patient in os.listdir(np_data_dir):
 if patient.endswith(".npy"):
   path_to_patient = os.path.join(np_data_dir,patient)
   patient_data = np.load(path_to_patient)
   hist_of_patient = patient_data[0][slice_nr].ravel()
   hist_list.append(hist_of_patient)
   number_of_total_slices += patient_data.shape[1]
   
print("The total number slices is: {}".format(number_of_total_slices))

In [ ]:
# Plot one example slice to ensure that the right data is loaded
plt.rcParams['figure.figsize'] = [10, 10]
plt.imshow(patient_data[0][0],cmap="gray")

In [ ]:
# 2. Plot histogram on a subset of slices

# Did a min-max normalization while creating the images
plt.rcParams['figure.figsize'] = [50, 50]
fig, axs = plt.subplots(10,10, figsize=(50, 50), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.2, wspace=.2)
axs = axs.ravel()

for i in range(100):
    axs[i].hist([hist_list[i]])
    axs[i].set_title(str(i))

In [ ]:
# 4. Get the image dimensions
patients =  get_list_of_patients(np_data_dir)
image_dimensions_x =[]
image_dimensions_y =[]
for patient in patients:
  arr = np.load(os.path.join(np_data_dir,patient+".npy"))
  image_shapes = arr.shape[0]
  image_dimensions_x.append(arr.shape[2])
  image_dimensions_y.append(arr.shape[3])


In [ ]:
# 5. Plot histogram of image dimensions 

fig = go.Figure()
fig.add_trace(go.Histogram(x=image_dimensions_x,name="X-dim"))
fig.add_trace(go.Histogram(x=image_dimensions_y,name="Y-dim"))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

fig.update_layout(
    title_text='Plot of dimensions', # title of plot
    xaxis_title_text='Value', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

In [ ]:
# 3. Choose the patch size
min_x = min(image_dimensions_x)
min_y = min(image_dimensions_y)
minimum_size = min(min_x,min_y)
next_min = math.floor(math.log(minimum_size,2),)
patch_size = (2**next_min, 2**next_min)


patch_size


In [ ]:

# Applying resizing to all data, because we don't want to handle training and dataset different.
if not os.path.isdir(np_preprocessed_data_dir):
  os.makedirs(np_preprocessed_data_dir)
  preproccessNp =Preprocess(patients,patch_size,0.5,np_preprocessed_data_dir)

else: 
  print("The directory: {} already exists, therefore the numpy array are not\
  created.\n If you want to load the data again delete\
  the directory: {}".format(np_data_dir,np_data_dir))

# Splitting Data
1.   Create Train,Test and Validation Set




In [ ]:
# 1.   Create Train,Test and Validation Set
dir = np_preprocessed_data_dir
patients =  get_list_of_patients(dir)
train_list=[]
val_list=[]
test_list= []
# From each category ["DCM","HCM","MINF","NOR","RV"] assign 80 % of the
# data to the trainset, and 10% to val, and test set each
for i in range(0,100,20):
  trainSub, valSub = get_split_deterministic(patients[i:(i+20)], fold=0, num_splits=5, random_state=100)  
  valSub,testSub = get_split_deterministic(valSub,fold=0,num_splits=2, random_state=100)
  train_list.extend(trainSub)
  val_list.extend(valSub)
  test_list.extend(testSub)


# Prepare the training


1.   Initiate ImageLoaser for train an validation
2.   Define transformations
3.   Initiate the MultiThreadedAugmenters
4. Plot histograms and images of one batch, and check if results are reasonable  
5. Plot images of one batch, and check if results are reasonable


**1.   Initiate ImageLoader for train an validation**


In [ ]:
batch_size =16
# seg_values =[1] # 
dataloader_train = ImageLoader2D(data=train_list, batch_size=batch_size, patch_size=patch_size,segmentation_value =1)
dataloader_validation =ImageLoader2D(data=val_list, batch_size=batch_size, patch_size=patch_size, segmentation_value =1)

**2. Define transformations**

In [ ]:
tr_transforms = get_train_transform(patch_size)

In [ ]:
val_transforms = get_val_transform(patch_size)

**3. Initiate and start MultiThreadedAugmenter**




In [ ]:
# Maybe need to adjust num_processes when aother server is used to execute the script
tr_gen = MultiThreadedAugmenter(dataloader_train, tr_transforms, num_processes=1,
                                num_cached_per_queue=1,
                                seeds=None, pin_memory=False)
val_gen = MultiThreadedAugmenter(dataloader_validation, val_transforms,
                                    num_processes=1, num_cached_per_queue=1,
                                    seeds=None,
                                    pin_memory=False)

In [ ]:
tr_gen.restart()

In [ ]:
val_gen.restart()

In [ ]:
batch = next(tr_gen)

**4. Plot images of one batch, and check if results are reasonable**

 **Batchviewer ist a tool that enables to quickly visualize a batch.  Through the visualization it was noticed that some slices were not labeled, these were removed accordingly in the preprocessing. It was not possible to install Batchviewer on Google colab, therefore it is not used here. Instead the images of one batch are plotted with matplotlib**

In [ ]:
plt.rcParams['figure.figsize'] = [20, 20]
fig, axs = plt.subplots(batch_size//4,4,figsize=(40,40))

fig.subplots_adjust(hspace = 0.2, wspace=.2)
axs = axs.ravel()
# Display mask for RV
for i,image in enumerate(batch["seg"]):
    axs[i].imshow(image[1],cmap="gray")
    axs[i].set_title(str(i))

**5. Plot histograms of one batch, and check if results are reasonable**

In [ ]:
batch = next(tr_gen)
plt.rcParams['figure.figsize'] = [10, 10]
fig, axs = plt.subplots(batch_size//2,2, figsize=(20, 20), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.2, wspace=.2)
axs = axs.ravel()
hist_list = []
for patient_slice in batch["data"]:
   hist_of_patient = patient_slice.ravel()
   hist_list.append(hist_of_patient)
for i in range(batch_size):
    axs[i].hist([hist_list[i]])
    axs[i].set_title(str(i))

# UNet


1.   **Build the model**
2.   **Callback Mehtod**



**1 .Build the model**

In [ ]:

keras.backend.clear_session()

num_classes =1
# Build model

img_size =patch_size+(num_classes,)
model = UNet(img_shape=img_size, out_ch=1, start_ch=64, depth=4, increasing_rate=2., activation='relu', 
		 dropout=0.5, batchnorm=True, maxpool=True, upconv=True, residual=False)
# model.summary()
def scheduler(epoch, lr):
  if epoch < 5:
    return lr
  else:
    return lr * tf.math.exp(-0.1)
model.summary()
opt = Adam(learning_rate =0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy')


**2. Callback function**

In [ ]:

def plotPred(model,val,shape,avgTrainLoss,avgValLoss):
  
  print("Epoch results")
  print("Average training loss over batches in epoch: {}".format(avgTrainLoss))
  print("Average validation loss over batches in epoch: {}".format(avgValLoss))
  for i in range(0,3):
    plt.subplot(3,3,(i*3)+1)
    pred = model.predict(val[0])
    pred = pred.reshape(shape)
    X_val = val[0].reshape(shape)
    y_val = val[1].reshape(shape)
    plt.imshow(X_val[i][0],cmap="gray")
    plt.title("Original")
    plt.subplot(3,3,(i*3)+2)
    plt.imshow(y_val[i][0],cmap="gray")
    plt.title("Ground Truth")
    plt.subplot(3,3,(i*3)+3)
    plt.imshow(pred[i][0],cmap="gray")
    plt.title("Predicted Segmentation")
    plt.tight_layout()
  plt.show()

# Training of the UNet


1.   Training
2.   Save model and training data
3.   Plot the results



**1. Training**

In [ ]:
# Not to pretty implemetation of the epochs, but i had no idea how 
# to write an traininggenerator for keras based on the batchgenerator
# The creation of the batches is random which means that it is possible that one slice,
# can be seen more than one time at each epoch and another is not presented to the model 
# in that epoch. In this case it isn't too much pain because over 20 Epochs one can assume
# that the random choose of the batch has not too much effet.
plt.rcParams['figure.figsize'] = [20, 10]
total_history = {"loss":[],"val_loss":[]}
epoch_history = {"loss":[],"val_loss":[]}
num_epochs = 20
num_batches_per_epoch =number_of_total_slices//batch_size
new_shape = (16,)+patch_size+(1,)
old_shape = (16,)+(1,)+patch_size
seg_channel = 1 #seg --> 0: Background, 1:RV cavity, 2: Myocardium, 3:LV-Cavity
for epoch in range(num_epochs):
  print("Started epoch: {}".format(epoch+1))
  for b in range(num_batches_per_epoch):
      batchTrain = next(tr_gen)
     
      X_train = np.reshape(batchTrain["data"][:,seg_channel],new_shape)
      y_train = np.reshape(batchTrain["seg"][:,seg_channel],new_shape)
      batchVal = next(val_gen)
      # Segmentation of right ventricel is in first channel
      X_val = np.reshape(batchVal["data"][:,seg_channel],new_shape)
      y_val = np.reshape(batchVal["seg"][:,seg_channel],new_shape)
      val = [X_val,y_val]
      history_of_batch = model.fit(x=X_train,y=y_train,validation_data=val,  
                                   epochs=1,batch_size=batch_size,verbose=0)
      epoch_history["loss"].extend(history_of_batch.history["loss"])
      epoch_history["val_loss"].extend(history_of_batch.history["val_loss"])
      
      
      if epoch ==0:
        val_1 = val
  avgTrainLoss = sum(epoch_history["loss"])/len(epoch_history["loss"]) 
  avgValLoss = sum(epoch_history["val_loss"])/len(epoch_history["val_loss"]) 
  plotPred(model,val_1,old_shape,avgTrainLoss,avgValLoss)
  
  total_history["loss"].append(avgTrainLoss)
  total_history["val_loss"].append(avgValLoss)
  epoch_history = {"loss":[],"val_loss":[]}
  history_path = os.path.join(root_dir,"/model/history.pkl")

**2. Save model and training data**

In [ ]:
model_path = os.path.join(root_dir,"model")
historyPath = os.path.join(root_dir,"/model/history.pkl")
os.makedirs(os.path.dirname(historyPath), exist_ok=True)


with open(history_path, 'wb') as f:
    pickle.dump(total_history, f)
with open(historyPath, 'rb') as f:
    total_history= pickle.load(f)

model.save(model_path)

In [ ]:
# model =keras.models.load_model("/content/drive/MyDrive/ADVMB/model")

In [ ]:
with open(historyPath, 'rb') as f:
    total_history= pickle.load(f)

**3. Plot the results**

In [ ]:
df = pd.DataFrame(total_history)


In [ ]:

fig=df.plot(labels=dict(value="binary cross entropy", index="epoch",x_ticks=[1,2]))
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    autosize=False,
    width=1000,
    height=500,

)

fig.show()



# Evaluation of the model
1. Make Prediction on the hole training set and plot a few examples

In [ ]:
def load_patient(patient):
      data = np.load(patient + ".npy")
      return data

In [ ]:
from batchgenerators.augmentations.normalizations import cut_off_outliers, mean_std_normalization, range_normalization, \
    zero_mean_unit_variance_normalization,min_max_normalization
patch_size=(128,128)
from medpy.metric.binary import dc
dices = []
threshold =0.8
plt.rcParams['figure.figsize'] = [20, 10]
imgOutputPath = os.path.join(root_dir,"images_RV")
os.makedirs(imgOutputPath, exist_ok=True)
new_shape =(128,128)

for i, patient in enumerate(test_list):
    patient_name =patient.split("/")[-1][0:10]
    data = load_patient(os.path.join(np_preprocessed_data_dir,patient_name))
    data = data[:,:,1,:,:]
    image = data[0] 
    segmentation = data[1]
    pred = image[None]
    pred = pred.reshape((image.shape[0],*patch_size,1))
    pred = model.predict(pred)
    pred = pred.reshape((1,image.shape[0],*patch_size))
    for index,mrt_slice in enumerate(image):
      
      if index ==5:
        plt.subplot(1,4,1)
        plt.title("MRT")
        plt.imshow(mrt_slice,cmap="gray")
        
        plt.subplot(1,4,2)
        plt.title("Segmentation")
        plt.imshow(segmentation[index],cmap ="gray")
        
        plt.subplot(1,4,3)
        plt.title("Prediction")
        plt.imshow(pred[0][index],cmap ="gray")
        
        threshold = 0.6
        pred[pred>=threshold] = 1
        pred[pred<threshold] =0
     
        dice =dc(pred[0][index],segmentation[index])
        plt.subplot(1,4,4)
        plt.title("Prediction with threshold, SDC: {}".format(round(dice,3)))
        plt.imshow(pred[0][index],cmap ="gray")
        
        # plt.savefig("/content/"+str(i)+"_"+str(index)+".pdf",bbox_inches='tight')
        plt.show()
      pred[pred>=threshold] = 1
      pred[pred<threshold] =0
# Because there are many unlabled mask after removing the mask for myo an LV the dice is only calculated when a mask is present
      
      if not np.sum(segmentation[index])==0:
         dices.append(dc(pred[0][index],segmentation[index]))
dices = np.array(dices)
mean =round(dices.mean(),5)
std =round(dices.std(),5)
print(mean,std)


In [ ]:

dices = []
for i, patient in enumerate(val_list):
    patient_name =patient.split("/")[-1][0:10]
    data = load_patient(os.path.join(np_preprocessed_data_dir,patient_name))
    data = data[:,:,1,:,:]
    image = data[0] 
    segmentation = data[1]
    pred = image[None]
    pred = pred.reshape((image.shape[0],*patch_size,1))
    pred = model.predict(pred)
    pred = pred.reshape((1,image.shape[0],*patch_size))
    for index,mrt_slice in enumerate(image):
      pred[pred>=threshold] = 1
      pred[pred<threshold] =0
# Because there are many unlabled mask after removing the mask for myo an LV the dice is only calculated when a mask is present
    
      if not np.sum(segmentation[index])==0:
        dices.append(dc(pred[0][index],segmentation[index]))
      
dices = np.array(dices)
mean =round(dices.mean(),5)
std =round(dices.std(),5)
print(mean,std)


In [ ]:
dices = []
for i, patient in enumerate(train_list):
    patient_name =patient.split("/")[-1][0:10]
    data = load_patient(os.path.join(np_preprocessed_data_dir,patient_name))
    data = data[:,:,1,:,:]
    image = data[0] 
    segmentation = data[1]
    pred = image[None]
    pred = pred.reshape((image.shape[0],*patch_size,1))
    pred = model.predict(pred)
    pred = pred.reshape((1,image.shape[0],*patch_size))
    for index,mrt_slice in enumerate(image):
      pred[pred>=threshold] = 1
      pred[pred<threshold] =0
# Because there are many unlabled mask after removing the mask for myo an LV the dice is only calculated when a mask is present
      if not np.sum(segmentation[index])==0:
        dices.append(dc(pred[0][index],segmentation[index]))
        
       
dices = np.array(dices)
mean =round(dices.mean(),5)
std =round(dices.std(),5)
print(mean,std)